In [11]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import pearsonr
import sys

In [13]:
import pandas as pd
import numpy as np

folder = 'birds/Eared Grebe Mar- May/'
files = glob.glob(folder + '*Bear*.txt')
files.sort()

df_all = pd.DataFrame(columns=['year', 'march_avg', 'april_avg', 'may_avg'])

for file in files:
    print(file)
    df = pd.read_table(file)
    i1 = df.index[df['Unnamed: 0'].eq('Average Count')]
    df = df[i1[0] + 1:]
    df = df[df['Week starting on...'].str.contains('checklist', na=False) == False]
    df.replace(0, np.nan, inplace=True)

    # Extract years from 'Week starting on...' column
    years = df['Week starting on...'].str.extract('(\d+)')[0].astype(float).dropna().unique().astype(int)    

    for year in years:
        # Filter data for the current year
        df_year = df[df['Week starting on...'].str.contains(str(year), na=False)] # Apply the fix here
        
        # Reshape data and calculate monthly averages
        df_melted = df_year.melt(id_vars=['Week starting on...'], var_name='date', value_name='count')
        df_melted['month'] = pd.to_numeric(df_melted['date'].str.extract('(\d+)/\d+')[0], errors='coerce').astype('Int64')
        monthly_avgs = df_melted.groupby('month')['count'].mean()

        # Store monthly averages in df_all
        df_all = pd.concat([df_all, pd.DataFrame({'year': [year],
                                                  'march_avg': [monthly_avgs.get(3, np.nan)],
                                                  'april_avg': [monthly_avgs.get(4, np.nan)],
                                                  'may_avg': [monthly_avgs.get(5, np.nan)]})])

# Convert df_all to xarray
df_all_xr = df_all.set_index('year').to_xarray()
df_all_xr = df_all_xr.rename_dims(year='time')
df_all_xr = df_all_xr.rename({'year': 'time'})

print(df_all_xr)

<xarray.Dataset> Size: 0B
Dimensions:    (time: 0)
Coordinates:
  * time       (time) object 0B 
Data variables:
    march_avg  (time) object 0B 
    april_avg  (time) object 0B 
    may_avg    (time) object 0B 


C:\Users\u1355\AppData\Local\Temp\ipykernel_34012\2918045469.py:39: UserWarning: rename 'year' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  df_all_xr = df_all_xr.rename({'year': 'time'})


In [17]:
df_all_df = df_all_xr.to_dataframe()

In [18]:
df_all_df

,march_avg,april_avg,may_avg
time,,,


In [16]:
df_all_df.to_csv('birds/EG_monthly_avgs/EG_Spring_Avgs.csv')

OSError: Cannot save file into a non-existent directory: 'birds\EG_monthly_avgs'